In [18]:
import duckdb
import pandas as pd
import numpy as np

In [36]:
def create_con():
    con = duckdb.connect(database=':memory:')
    return con

def execute_local(con, query):
    result = con.execute(query).fetchdf()
    return result

con = create_con()
q = '''
        CREATE TABLE patients AS SELECT * FROM read_csv_auto('Data/synthetic/csv/patients.csv')
    '''
execute_local(con, q)
patients = execute_local(con, '''select * from patients''')

df1 = patients.where(pd.notnull(patients), None)
df2 = df1.where(pd.notna(df1), None)
df2


,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,b8d8578a-3403-a5c5-6e44-19cae89e66e6,1996-09-18,NaT,999-63-4686,S99918900,X11471625X,Mr.,Josiah310,Marks830,None,...,Lelystad Flevoland NL,200 Reynolds Pathway,Lelystad,Flevoland,Lelystad,8241,52.535897,5.506974,0.0,28196.83
1,6f196958-2892-9208-d039-3bc0da53e441,1967-11-10,NaT,999-78-7949,S99918013,X21477417X,Mrs.,Chanel977,Corkery305,None,...,Lelystad Flevoland NL,470 Spencer View Suite 70,Emmeloord,Flevoland,Emmeloord,8303,52.721992,5.813659,0.0,401884.43
2,16b4683a-7530-6f59-8b55-55c2ff94cb4a,1981-12-03,NaT,999-66-8820,S99992638,X61515580X,Mr.,Shelton25,Kirlin939,None,...,Lelystad Flevoland NL,1057 Von Orchard Unit 94,Urk,Flevoland,Urk,8321,52.635535,5.612077,0.0,54141.90
3,30043951-c45e-bd62-73bf-33416a271dea,1997-07-16,NaT,999-22-6918,S99927851,X19029264X,Ms.,Marcia467,Ryan260,None,...,Lelystad Flevoland NL,754 Grady Overpass,Zeewolde,Flevoland,Zeewolde,3897,52.398065,5.459499,0.0,54159.71
4,03b2c661-cd30-40a7-2afd-d600d07533da,1966-01-09,NaT,999-31-6521,S99976066,X81949173X,Mr.,Michel472,Berge125,None,...,Emmeloord Flevoland NL,580 Fahey Spur Suite 8,Ens,Flevoland,Ens,8307,52.642471,5.837429,0.0,96259.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,122adea7-6afd-81cb-2162-185263fa7ce9,1956-03-21,NaT,999-72-2996,S99933944,X25817725X,Mr.,Antonia30,Breitenberg711,None,...,Emmeloord Flevoland NL,262 Moore Branch Suite 78,Urk,Flevoland,Urk,8321,52.620047,5.588224,0.0,137154.59
532,ba345a55-3a6d-8884-fa60-41d41375d2fb,1988-12-24,NaT,999-25-2501,S99982325,X80986133X,Ms.,Trina350,Schmeler639,None,...,Lelystad Flevoland NL,394 Davis Rapid,Lelystad,Flevoland,Lelystad,8211,52.548077,5.471362,0.0,949358.17
533,6cebf2c8-2017-4e8a-3bff-d15e62ba8a3b,1970-01-18,NaT,999-15-9182,S99931227,X61523423X,Mrs.,Leena55,Ebert178,None,...,Dronten Flevoland NL,907 Ziemann Well Apt 57,Emmeloord,Flevoland,Emmeloord,8302,52.665713,5.737818,0.0,621971.69
534,6e3e5e33-c0bd-6916-7b6f-d671e1296a3d,1979-01-24,NaT,999-85-7103,S99914452,X17805573X,Mrs.,Cleopatra935,Schmitt836,None,...,Emmeloord Flevoland NL,951 Smitham Terrace Apt 74,Emmeloord,Flevoland,Emmeloord,8302,52.664298,5.698275,0.0,414181.63


In [40]:
def isKAnonymized(df, k):
    for index, row in df.iterrows():
        query = ' & '.join([f'{col} == {row[col]}' for col in df.columns])
        rows = df.query(query)
        if rows.shape[0] < k:
            return False
    return True

def validate_k_anonimity(df):
    count = 0
    for i in range(len(df)):
        for j in range(len(df)):
            if i == j:
                continue
            current = df.iloc[i]
            if distinguishable(current, df.iloc[j]):
                count = count + 1
    print(count)

def generalize(df, depths):
    return df.apply(lambda x: x.apply(lambda y: int(int(y/(10**depths[x.name]))*(10**depths[x.name]))))
    
def make_k_anonymous(df, row_preservation=0.5):
    pass


validate_k_anonimity(zips)

90


In [58]:
def generalize(df, depths):
    return df.apply(lambda x: x.apply(lambda y: int(int(y/(10**depths[x.name]))*(10**depths[x.name]))))

depths = {
    'ZIP': 1
}


output = generalize(df2[['ZIP']], depths)

output = pd.concat([output, df2[['BIRTHDATE']]], axis=1, join="inner")

isKAnonymized(output, 4)

SyntaxError: Python keyword not valid identifier in numexpr query (<unknown>, line 1)